# Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import algorithms.bernstein_vazirani as bv
import algorithms.qft as qft

from qiskit import QuantumCircuit, transpile, Aer
from qiskit_ibm_provider import IBMProvider
from qiskit.tools.monitor import job_monitor
from qiskit.quantum_info.analysis import hellinger_fidelity

# Parameters

In [2]:
def qiskit_opt_0(qc: QuantumCircuit, backend):
    return transpile(qc, backend, optimization_level=0)

def qiskit_opt_3(qc: QuantumCircuit, backend):
    return transpile(qc, backend, optimization_level=3)

def qiskit_opt_3_sabre(qc: QuantumCircuit, backend):
    return transpile(qc, backend, optimization_level=3, layout_method='sabre', routing_method='sabre')

In [3]:
min_qubits, max_qubits = 4, 7
algorithms = { 'Bernstein Vazirani': bv, 'QFT': qft }
techniques = { 
    'Qiskit Opt 0': qiskit_opt_0, 
    'Qiskit Opt 3': qiskit_opt_3, 
    'Qiskit Opt 3 Sabre': qiskit_opt_3_sabre 
}

In [4]:
provider = IBMProvider()
backend = provider.get_backend("ibm_oslo")

In [5]:
from qiskit.providers.fake_provider import FakeOslo
fake_backend = FakeOslo()

# Create Transpiled Circuits

In [6]:
qcs = []
tqcs = []
cx_counts = []

for al_name, al in algorithms.items():
    for n in range(min_qubits, max_qubits+1):
        qc = al.test(n)
        qcs.append(qc)
        for t_name, t in techniques.items():
            tqc = t(qc, backend)
            tqcs.append(tqc)
            cx_counts.append(tqc.count_ops()['cx'])

# Simulate QCs

In [7]:
def simulate_classically(qc_list):
    aer_sim = Aer.get_backend('aer_simulator')
    tqc_list = transpile(qc_list, aer_sim)
    job_sim = aer_sim.run(tqc_list, shots=1000)
    results = job_sim.result()
    return results

In [8]:
sim_res_counts = simulate_classically(qcs).get_counts()
display(sim_res_counts)

[{'111': 1000},
 {'1111': 1000},
 {'11111': 1000},
 {'111111': 1000},
 {'0000': 1000},
 {'00000': 1000},
 {'000000': 1000},
 {'0000000': 1000}]

# Run on IBM QC

In [9]:
# job = backend.run(tqcs, shots=1000)
job = fake_backend.run(tqcs, shots=1000)

In [10]:
# job_monitor(job, interval=2)

In [11]:
def calculate_avg_cx_errors(backend):
    cx_gates = [g for g in backend.properties().gates if g.gate == 'cx']
    cx_error_rate = 0
    for g in cx_gates:    
        cx_error_rate += g.parameters[0].value

    avg_cx_error_rate = cx_error_rate /len(cx_gates)
    return avg_cx_error_rate

In [12]:
avg_cx_error_rate = calculate_avg_cx_errors(backend)
print('avg_cx_error_rate: ', avg_cx_error_rate)

avg_cx_error_rate:  0.008230401598889053


In [13]:
qc_res_counts = job.result().get_counts()

# Check Fidelities

In [31]:
n_s = max_qubits-min_qubits+1

for al_index, al_name in enumerate(algorithms.keys()):
    for n_index in range(n_s):
        n = n_index + min_qubits
        print('{}: n={}'.format(al_name, n))
        sim_res_counts_n = sim_res_counts[al_index*(n_s) + n_index]
        for t_index, t_name in enumerate(techniques.keys()):
            counts_index = al_index*(n_s)*len(techniques) + n_index*len(techniques) + t_index
            qc_counts = qc_res_counts[counts_index]
            cx_count = cx_counts[counts_index]
            hf = hellinger_fidelity(sim_res_counts_n, qc_counts)
            print('{}: # cx gates: {}, fidelity={}'.format(t_name, cx_count, hf))
        print('---')
    print('------')
            

Bernstein Vazirani: n=4
Qiskit Opt 0: # cx gates: 9, fidelity=0.9030000000000001
Qiskit Opt 3: # cx gates: 3, fidelity=0.949
Qiskit Opt 3 Sabre: # cx gates: 3, fidelity=0.9210000000000002
---
Bernstein Vazirani: n=5
Qiskit Opt 0: # cx gates: 25, fidelity=0.8039999999999999
Qiskit Opt 3: # cx gates: 7, fidelity=0.909
Qiskit Opt 3 Sabre: # cx gates: 7, fidelity=0.8909999999999999
---
Bernstein Vazirani: n=6
Qiskit Opt 0: # cx gates: 14, fidelity=0.833
Qiskit Opt 3: # cx gates: 7, fidelity=0.866
Qiskit Opt 3 Sabre: # cx gates: 9, fidelity=0.8530000000000001
---
Bernstein Vazirani: n=7
Qiskit Opt 0: # cx gates: 21, fidelity=0.774
Qiskit Opt 3: # cx gates: 10, fidelity=0.82
Qiskit Opt 3 Sabre: # cx gates: 8, fidelity=0.8699999999999999
---
------
QFT: n=4
Qiskit Opt 0: # cx gates: 30, fidelity=0.8240000000000001
Qiskit Opt 3: # cx gates: 25, fidelity=0.809
Qiskit Opt 3 Sabre: # cx gates: 25, fidelity=0.789
---
QFT: n=5
Qiskit Opt 0: # cx gates: 65, fidelity=0.668
Qiskit Opt 3: # cx gates: 4

In [15]:
sim_res_counts[-4]

{'0000': 1000}

In [16]:
al_index = 1
n_index = 0
t_index = 2

display(qc_res_counts[al_index*(n_s)*len(techniques) + n_index*len(techniques) + t_index])

{'1010': 3,
 '0001': 39,
 '0000': 789,
 '1000': 45,
 '0010': 36,
 '0100': 22,
 '0111': 3,
 '1100': 15,
 '1110': 15,
 '0011': 4,
 '1001': 14,
 '1111': 5,
 '0110': 7,
 '0101': 2,
 '1011': 1}

In [17]:
hf = hellinger_fidelity(sim_res_counts[-4], qc_res_counts[al_index*(n_s)*len(techniques) + n_index*len(techniques) + t_index])
display(hf)

0.789